In [1]:
import json
import re
import os
from functions import *

# Disable verification warning when accessing GSO site
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
# Set to True if new reports are updated
get_report_url = False

# Set to True to check if the values are appropriate, False to export them as csv, and combine them into one Excel file
check_values = False

download_path = os.path.join(os.getcwd(), "raw_xlsx")

if get_report_url == True:
    all_reports_url = crawl_url()
else:
    with open("all_reports_url.pkl", "rb") as f:
        all_reports_url = pickle.load(f)

test = all_reports_url[0:24] # Sample to test

In [ ]:
error = download_data(download_path, all_reports_url)

# # Save error urls
# with open("error_url.pkl", "wb") as write:
#     pickle.dump(error, write)

remove_duplicates(download_path)

In [52]:
'''
There are 3 types of dataformats:
- Quarterly reports: include data of Mar, Jun, Sep, Dec
- January reports
- Other months
'''
quarterly_files = []
monthly_files = []
january_files = []

for excel_path in os.listdir(download_path):
    if excel_path.endswith(".xlsx") or excel_path.endswith('.xls'):
        if any(q in excel_path for q in ["Q", "-3", "03", "-6", "06", "09", "-9", "12", "T3", "T6", "T9", "T12"]):
            quarterly_files.append(excel_path)
        elif any(m in excel_path for m in ["-01", "T01"]):
            january_files.append(excel_path)
        else:
            monthly_files.append(excel_path)
    else: 
        pass

In [5]:
with open("non_excel_reports.pkl", "rb") as read:
    non_excel = pd.read_pickle(read)

In [53]:
# Check if data have the same sheet names
# Quarterly data and January data files are formatted differently, so they will be separated into a different dataset
def check_columns(name_list= [quarterly_files, monthly_files, january_files]):
    file_error = []
    sheet_names = []
    for file in tqdm(name_list):
        excel = os.path.join("raw_xlsx", file)
        try:
            if file.endswith('.xlsx'):
                sheet_names.append(pd.ExcelFile(excel, engine="openpyxl").sheet_names)
            elif file.endswith(".xls"):
                sheet_names.append(pd.ExcelFile(excel, engine="xlrd").sheet_names)
        except Exception as error:
            file_error.append(file)
    data = pd.DataFrame(sheet_names).transpose()
    column_names = [name for name in name_list if name not in file_error]
    try:
        data.columns = column_names
    except Exception as error:
        file_error.append(file)
    return data, file_error

def combine_columns(df: pd.DataFrame, n_columns: int):  
    df = df.rename(columns={
        0:"name"
    })
    for i in range(0, n_columns - 1):
        df = df.assign(
            name = lambda df : df.loc[:, "name"].combine_first(df[i+1])
        )
    return df

In [54]:
# Check for files incompatabilities and unfit number of column
j_check, jan_error = check_columns(january_files)
m_check, monthly_error = check_columns(monthly_files)


100%|██████████| 58/58 [00:10<00:00,  5.33it/s]


In [64]:
m_check

,02-TABLE-02-2022.xlsx,02-TABLE-02-2023-1.xlsx,02-TABLE-04-2023.xlsx,02-TABLE-05-2023.xlsx,02-TABLE-07-2023.xlsx,02-TABLE-08-2023.xlsx,02-TABLE-08-2024.xlsx,02-TABLE-10-2021.xlsx,02-TABLE-10-2022.xlsx,02-TABLE-10-2023.xlsx,...,Bieu-04-2018.En_.xlsx,Bieu-4-2017.xlsx,Bieu-5.2017.xlsx,Bieu-7.2017-1.xlsx,Bieu-8-2017-1.xlsx,Bieu-T10-2017.En_.xlsx,Table-02-2025-change.xlsx,TABLE-04-2025.xlsx,TABLE-05.2025.xlsx,TABLE-T11-2024.xlsx
0,1. Agricultural,1. Agricultural,1. Agricultural,1. Agricultural,1. Agricultural,1. Agricultural,1. Agricultural,1.Agriculture,1.Agriculture,1.Agriculture,...,01NN,NN,01NN (2),01NN,01NN,01NN,1. Agricultural,1. Agricultural,1. Agricultural,1.Agriculture
1,2 IIP,2.IIP,2.IIP,2.IIP,2.IIP,2. IIP,2. IIP,2.IIP,2.IIP,2.IIP,...,IIP,IIP,IIP,IIP,IIP,IIP,2.IIP,2.IIP,2.IIP,2.IIPthang
2,3 Industrial product,3.Industrial product,3. Key indus products,3. Key indus products,3 Industrial product,3. Industrial product,3. Industrial product,3. Industrial product,3. Industrial product,3. Industrial product,...,SP,SP,SP,SP,SP,SP,3.Industrial product,3.Industrial product,3.Industrial product,3. Industrial product
3,4 LEI,4. LEI,4.LEI,4.LEI,4 LEI,4. LEI,4. LEI,4. LEI,4. LEI,4. LEI,...,LAO DONG,CS TT TK,CS TT TK,CS TT TK,CS TT TK,CS TT TK,4. LEI,4. LEI,4. LEI,4. LEI
4,5 LEI province,5. LEI province,5. LEI province,5. LEI province,5 LEI province,5. LEI province,5. LEI province,5. LEI province,5. LEI province,5. LEI province,...,DN,LAO DONG,LAO DONG,LAO DONG,LAO DONG,LAO DONG,5. LEI province,5. LEI province,5. LEI province,5. LEI province
5,6 Enterprise Indicators,6 Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6 Enterprise Indicators,6. Enteprise Indicators,...,DN1,DN1,DN1,Sheet1,DN1,DN1,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators
6,7 Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7 Newly regis Enter,7.Newly regis Enter,...,DN2,DN2,DN2,DN1,DN2,DN2,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter
7,8 Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,8 Enter returned,8. Enter returned,...,VonDT,VonDT,VonDT,DN2,VonDT,VonDT,8. Enter returned,8. Enter returned,8. Enter returned,8. Enter returned
8,9 Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9 Temporarily ceased,9. Temorarily ceased,...,05DTNN,FDI,05DTNN,VonDT,05DTNN,05DTNN,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased
9,10 Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10 Completed dissolution,10. Comleted dissolution,...,Tongmuc,tongmuc-OK,tongmuc-OK,05DTNN,Sheet1,Tongmuc,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution


In [56]:
j_check

,02-TABLE-01-2021.xlsx,02-TABLE-01-2022.xlsx,02-TABLE-01-2023-1.xlsx,02-TABLE-01-2024.xlsx,02.TABLE-01-2025.xlsx,Bieu-01-2018.En_.xlsx,Bieu-01.2017.En_.xlsx
0,1 agricultural,1 Agriculture,1. Agriculture,1. Agriculture,1. Agriculture,01NN,01NN
1,2 IIP,2 IIP,2. IIP,2. IIP,2. IIP,IIP,IIP
2,3 industrial products,3 Industrial product,3. Industrial product,3. Industrial product,3. Industrial product,SP,SP
3,4 LEI,4 LEI,4. LEI,4. LEI,4. LEI,LAO DONG,CS TT TK
4,5. LEI by province,5 LEI province,5. LEI province,5. LEI province,5. LEI province,DN1,LAO DONG
5,6 indicators of enterprises,6 Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,6. Enterprise Indicators,DN1 2,DN1
6,7 newly registered enterprises,7 Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,7. Newly regis Enter,DN2 3,DN2
7,8. enterprises returned to oper,8 Enter returned,8. Enter returned,8. Enter returned,8. Enter returned,VonDT,VonDT
8,9 temporarily ceased enterprise,9 Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,9. Temporarily ceased,05DTNN,FDI
9,10. enterprise dissolution,10 Completed dissolution,10. Completed dissolution,10. Completed dissolution,10. Completed dissolution,Tongmuc,Tongmuc


In [ ]:
# This have to be adjusted manually
jan_anomalies = ['Bieu-01-2018.En_.xlsx', 'Bieu-01.2017.En_.xlsx']

monthly_anomalies = [
    'Bieu-02.2017.En_.xls', 
    'Bieu-4-2017.xlsx',
    'Bieu-5.2017.xlsx', 
    'Bieu-7.2017-1.xlsx', 
    'Bieu-8-2017-1.xlsx',
    'Bieu-T10-2017.En_.xlsx',
    'BC-so-lieu-t10-2019.xlsx',
    'Bieu-04-2018.En_.xlsx', 
] 

In [82]:
jan_error.extend(jan_anomalies)
monthly_error.extend(monthly_anomalies)

In [85]:

valid_jan_files = [x for x in january_files if x not in jan_error]
print(f"Percentage of valid January files: {
    round(len(valid_jan_files)/len(january_files)*100, 2)
}%")

valid_monthly_files = [x for x in monthly_files if x not in monthly_error]
print(f"Percentage of valid Monthly files: {
    round(len(valid_monthly_files)/len(monthly_files)*100, 2)
}%")

Percentage of valid January files: 55.56%
Percentage of valid Monthly files: 53.45%


In [ ]:
# Process January data
number_of_sheets = 19

with open("sheetnames.json") as js:
    sheet_cols = json.load(js)

# Process January data
for sheet_index in tqdm(range(0, number_of_sheets)):
        combine_df = pd.DataFrame()

        if sheet_index == 15:
            row_adj = 1
        else:
            row_adj = 0
        for excel_path in valid_jan_files:
            print(f"Processing {excel_path}")
            sheet = use_columns(excel_path, sheet_index).iloc[1 + row_adj:,:]
            combine_df = pd.concat([combine_df, sheet], axis = 0)

        # Custom conditions
        if sheet_index in [0, 10, 11, 13, 14, 15, 16, 17, 18]: # Sheets that need to merge multiple name columns to make sense
            if sheet_index == 15:
                ncol = 3
                bad_label = combine_df.iloc[:,1].astype(str).str.contains(r"Of which:", regex = True, flags = re.IGNORECASE)
                combine_df.loc[bad_label, 1] = None 
            else:
                ncol = 2
            combine_df = combine_columns(combine_df, ncol)
            var_list = ["month", "name"]
        else:
            var_list = ["month"]
        if sheet_index == 2:
            adjust = 1
        else:
            adjust = 0

        var_list.extend(list(sheet_cols["january"][f"{sheet_index}"]["columns"].values()))

        combine_df = combine_df.rename(columns=dict(
            list(zip(
                list(map(int, sheet_cols["january"][f"{sheet_index}"]["columns"].keys())),
                list(sheet_cols["january"][f"{sheet_index}"]["columns"].values())
            ))
        ))[var_list] # Rearranging the columns

        if check_values == False:
            combine_df = clean_data(
                combine_df,
                var_list[1],
                combine_df.columns[2 + adjust:]
            )
            combine_df.to_csv(os.path.join("combined_data", "january_data", f"{sheet_cols["monthly"][f"{sheet_index}"]["sheet"]}.csv"), index=False)


In [ ]:
# Combine into 1 Excel file
if check_values == False:
    with pd.ExcelWriter(
        os.path.join("combined_data", "january_data", "january_macro_data.xlsx"), 
        engine = "openpyxl"
    ) as writer:
        for csv_file in tqdm(os.listdir(os.path.join("combined_data", "january_data"))):
            if csv_file.endswith(".csv"):
                df = pd.read_csv(os.path.join("combined_data", "january_data", csv_file), index_col=False)
                sheet_name = os.path.splitext(csv_file)[0]
                df.to_excel(writer, sheet_name = sheet_name, index = False)

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:00<00:00, 35.56it/s]


In [89]:
# Process monthly data
check_values = False

with open("sheetnames.json") as js:
    sheet_cols = json.load(js)

# Create a function to streamline the process

number_of_sheets = 19

# Process monthly data
for sheet_index in tqdm(range(0, number_of_sheets)):
    combine_df = pd.DataFrame()

    if sheet_index == 15:
        row_adj = 1
    else:
        row_adj = 0
    for excel_path in valid_monthly_files:
        try:
            sheet = use_columns(excel_path, sheet_index).iloc[1 + row_adj:,:]
            combine_df = pd.concat([combine_df, sheet], axis = 0)
        except Exception as error:
            print(f"Error at file {excel_path}:")
            print(error)

    # Custom conditions
    if sheet_index in [0, 10, 11, 13, 14, 15, 16, 17, 18]: # Sheets that need to merge multiple name columns to make sense
        if sheet_index == 15:
            ncol = 3
            bad_label = combine_df.iloc[:,1].astype(str).str.contains(r"Of which:", regex = True, flags = re.IGNORECASE)
            combine_df.loc[bad_label, 1] = None 
        else:
            ncol = 2
        if sheet_index in [16, 17]:
            is_parent = combine_df.iloc[:,0].str.startswith("By ", na=False)
            combine_df["name"] = (
                (combine_df.iloc[:,0].where(is_parent).ffill()).astype(str) + 
                " - " + 
                combine_df.iloc[:,0].astype(str)
            ).str.replace(r"nan", "", regex=True).replace(r"^\s-\s", "", regex=True).replace("", None)
        else:
            combine_df = combine_columns(combine_df, ncol)
        var_list = ["month", "name"]
    else:
        var_list = ["month"]
    if sheet_index == 2:
        adjust = 1
    else:
        adjust = 0

    var_list.extend(list(sheet_cols["monthly"][f"{sheet_index}"]["columns"].values()))

    combine_df = combine_df.rename(columns=dict(
        list(zip(
            list(map(int, sheet_cols["monthly"][f"{sheet_index}"]["columns"].keys())),
            list(sheet_cols["monthly"][f"{sheet_index}"]["columns"].values())
        ))
    ))[var_list] # Rearranging the columns

    if check_values == False:
        combine_df = clean_data(
            combine_df,
            var_list[1],
            combine_df.columns[2 + adjust:]
        )
        combine_df.to_csv(
            os.path.join(
                "combined_data", 
                "monthly_data", 
                f"{sheet_cols["monthly"][f"{sheet_index}"]["sheet"]}.csv"
            ), index=False
        )

100%|██████████| 19/19 [04:19<00:00, 13.66s/it]


In [96]:
# Combine into 1 Excel file
if check_values == False:
    with pd.ExcelWriter(
        os.path.join("combined_data", "monthly_data", "monthly_macro_data.xlsx"), 
        engine = "openpyxl"
    ) as writer:
        for csv_file in tqdm(
            os.listdir(os.path.join("combined_data", "monthly_data"))
        ):
            if csv_file.endswith(".csv"):
                df = pd.read_csv(os.path.join("combined_data", "monthly_data", csv_file), index_col=False)
                sheet_name = os.path.splitext(csv_file)[0]
                df.to_excel(writer, sheet_name = sheet_name, index = False)

100%|██████████| 21/21 [00:00<00:00, 26.84it/s]


### Legacy code

In [ ]:
# # Agriculture
# combined_agriculture = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, 0)
#     combined_agriculture = pd.concat([combined_agriculture, sheet], axis = 0)

# combined_agriculture.iloc[:,0] = combined_agriculture.iloc[:,0].ffill()

# combined_agriculture = combined_agriculture.assign(
#     product = combined_agriculture.iloc[:,0].astype(str) + " - " + combined_agriculture.iloc[:,1].fillna("")
# ).rename(columns={
#     2:"thousand_ha_yoy",
#     3:"thousand_ha_current",
#     4:"pct_yoy"
# })[["month", "product","thousand_ha_yoy", "thousand_ha_current", "pct_yoy"]] # Rearranging the columns

# combined_agriculture = clean_data(
#     combined_agriculture,
#     "product",
#     combined_agriculture.columns[2:]
# )

# if check_values == False:
#     combined_agriculture.to_csv("combined_data/combined_agr.csv", index=False)

In [ ]:
# # IIP

# combined_iip = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, 1)
#     combined_iip = pd.concat([combined_iip, sheet], axis = 0)
# combined_iip = combined_iip.rename(columns={
#     0:"industry",
#     1:"pct_yoy_lag1",
#     2:"pct_mom",
#     3:"pct_yoy",
#     4:"accum_yoy"
# })[["month", "industry", "pct_yoy_lag1", "pct_mom", "pct_yoy", "accum_yoy"]]

# if check_values == False:
#     combined_iip = clean_data(
#         combined_iip, 
#         "industry", 
#         combined_iip.columns[2:]
#     )
#     combined_iip.to_csv("combined_data/combined_iip.csv", index=False)

# clean_data(
#         combined_iip, 
#         "industry", 
#         combined_iip.columns[2:]
#     )

In [ ]:
# # Key Industrial products
# combined_kip = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 2)
#     combined_kip = pd.concat([combined_kip, sheet], axis = 0)

# # Rearrange "month" to be the first column
# combined_kip = combined_kip.rename(columns={
#     0:"product",
#     1:"unit",
#     2:"performance_lag1",
#     3:"est_performance",
#     4:"accum_performance",
#     5:"pct_yoy",
#     6:"pct_accum_yoy"
# })[["month", "product", "unit", "performance_lag1", "est_performance", "accum_performance", "pct_yoy", "pct_accum_yoy"]]

# if check_values == False:
#     combined_kip = clean_data(
#         combined_kip, 
#         "product", 
#         combined_kip.columns[3:]
#     )
#     combined_kip.to_csv("combined_data/combined_kip.csv", index=False)

# clean_data(
#     combined_kip, 
#     "product", 
#     combined_kip.columns[3:]
# )

In [ ]:
# # Labour Employed Index - LEI - BY INDUSTRY
# combined_lei_ind = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 3)
#     combined_lei_ind = pd.concat([combined_lei_ind, sheet], axis = 0)

# combined_lei_ind = combined_lei_ind.rename(columns={
#     0:"industry",
#     1:"ind_lei_mom",
#     2:"ind_lei_yoy"
# })[["month", "industry", "ind_lei_mom", "ind_lei_yoy"]]

# if check_values == False:
#     combined_lei_ind = clean_data(
#         combined_lei_ind,
#         "industry",
#         combined_lei_ind.columns[2:]
#     )
#     combined_lei_ind.to_csv("combined_data/combined_lei_ind.csv", index=False)

# clean_data(
#     combined_lei_ind,
#     "industry",
#     combined_lei_ind.columns[2:]
# )

In [ ]:
# # Labour Employed Index - LEI - BY PROVINCE
# combined_lei_prov = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 4)
#     combined_lei_prov = pd.concat([combined_lei_prov, sheet], axis = 0)

# combined_lei_prov = combined_lei_prov.rename(columns={
#     0:"province",
#     1:"prov_lei_mom",
#     2:"prov_lei_yoy"
# })[["month", "province", "prov_lei_mom", "prov_lei_yoy"]]

# if check_values == False:
#     combined_lei_prov = clean_data(
#         combined_lei_prov,
#         "province",
#         combined_lei_prov.columns[2:]
#     )
#     combined_lei_prov.to_csv("combined_data/combined_lei_prov.csv", index=False)
# clean_data(
#     combined_lei_prov,
#     "province",
#     combined_lei_prov.columns[2:]
# )

In [ ]:
# # Some indicators about enterprise

# combined_ei = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 5)
#     combined_ei = pd.concat([combined_ei, sheet], axis = 0)

# combined_ei = combined_ei.rename(columns={
#     0: "indicator",
#     1: "value_lag1",
#     2: "value",
#     3: "accum_value",
#     4: "value_mom",
#     5: "value_yoy",
#     6: "accum_yoy"
# })[["month", "indicator", "value_lag1", "value", "accum_value", "value_mom", "value_yoy", "accum_yoy"]]

# if check_values == False:
#     combined_ei = clean_data(
#         combined_ei,
#         "indicator",
#         combined_ei.columns[2:]
#     )
#     combined_ei.to_csv("combined_data/combined_ei.csv", index=False)

# clean_data(
#     combined_ei,
#     "indicator",
#     combined_ei.columns[2:]
# )

In [ ]:
# # New Registered Enterprises

# combined_new_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 6).drop(0, axis=1)
#     combined_new_e = pd.concat([combined_new_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_new_e = combined_new_e.rename(columns={
#     1: "industry",
#     2: "accum_new_enterprise",
#     3: "accum_capital_bil",
#     4: "accum_employee",
#     5: "accum_new_yoy",
#     6: "accum_cap_yoy",
#     7: "accum_emp_yoy"
# })[[
#     "month",
#     "industry",
#     "accum_new_enterprise",
#     "accum_capital_bil",
#     "accum_employee",
#     "accum_new_yoy",
#     "accum_cap_yoy",
#     "accum_emp_yoy"
# ]]

# if check_values == False:
#     combined_new_e = clean_data(
#         combined_new_e,
#         "industry",
#         combined_new_e.columns[2:]
#     )
#     combined_new_e.to_csv("combined_data/combined_new_e.csv", index=False)

# clean_data(
#     combined_new_e,
#     "industry",
#     combined_new_e.columns[2:]
# )

In [ ]:
# # Enterprises returned

# combined_return_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 7)
#     combined_return_e = pd.concat([combined_return_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_return_e = combined_return_e.rename(columns={
#     0: "industry",
#     1: "accum_return_ent_yoy",
#     2: "accum_return_ent",
#     3: "return_ent_pct_yoy"
# })[["month", "industry", "accum_return_ent_yoy", "accum_return_ent", "return_ent_pct_yoy"]]

# if check_values == False:
#     combined_return_e = clean_data(
#         combined_return_e,
#         "industry",
#         combined_return_e.columns[2:]
#     )
#     combined_return_e.to_csv("combined_data/combined_return_e.csv", index=False)

# clean_data(
#     combined_return_e,
#     "industry",
#     combined_return_e.columns[2:]
# )

In [ ]:
# # Enterprises temporary ceased

# combined_ceased_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 8)
#     combined_ceased_e = pd.concat([combined_ceased_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_ceased_e = combined_ceased_e.rename(columns={
#     0: "industry",
#     1: "accum_ceased_ent_yoy",
#     2: "accum_ceased_ent",
#     3: "ceased_ent_pct_yoy"
# })[["month", "industry", "accum_ceased_ent_yoy", "accum_ceased_ent", "ceased_ent_pct_yoy"]]

# if check_values == False:
#     combined_ceased_e = clean_data(
#         combined_ceased_e,
#         "industry",
#         combined_ceased_e.columns[2:]
#     )
#     combined_ceased_e.to_csv("combined_data/combined_ceased_e.csv", index=False)

# clean_data(
#     combined_ceased_e,
#     "industry",
#     combined_ceased_e.columns[2:]
# )

In [ ]:
# # Enterprises complete dissolution

# combined_diss_e = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 9)
#     combined_diss_e = pd.concat([combined_diss_e, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_diss_e = combined_diss_e.rename(columns={
#     0: "industry",
#     1: "accum_diss_ent_yoy",
#     2: "accum_diss_ent",
#     3: "diss_ent_pct_yoy"
# })

# if check_values == False:
#     combined_diss_e = clean_data(
#         combined_diss_e,
#         "industry",
#         combined_diss_e.columns[2:]
#     )
#     combined_diss_e.to_csv("combined_data/combined_diss_e.csv", index=False)

In [ ]:
# # Realized investment under State budget

# combined_invest = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 10, col_index = [1, 2, 3, 4, 5, 6])
#     combined_invest = pd.concat([combined_invest, sheet], axis = 0)

# combined_invest = combined_invest.rename(columns={
#     1:"area",
#     2:"inv_bil_lag1",
#     3:"inv_bil",
#     4:"accum_inv",
#     5:"accum_inv_to_plan",
#     6:"accum_inv_yoy"
# })[["month", "area", "inv_bil_lag1", "inv_bil", "accum_inv", "accum_inv_to_plan", "accum_inv_yoy"]]

# if check_values == False:
#     combined_invest = clean_data(
#         combined_invest,
#         "area",   
#         combined_invest.columns[2:]
#     )
#     combined_invest.to_csv("combined_data/combined_invest.csv", index=False)

# clean_data(
#     combined_invest,
#     "area",
#     combined_invest.columns[2:]
# )

In [ ]:
# # FDI

# combined_fdi = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 11, col_index = [1, 2, 3, 4])
#     combined_fdi = pd.concat([combined_fdi, sheet], axis = 0)

# # This data does not seem to have monthly data, only accummulative data
# combined_fdi = combined_fdi.rename(columns={
#     1: "area",
#     2: "n_projects",
#     3: "new_registered_cap_mil_usd",
#     4: "reg_cap_adjust_mil_usd"
# }).assign(
#     n_projects = lambda df : df["n_projects"].fillna(0),
#     new_registered_cap_mil_usd = lambda df : df["new_registered_cap_mil_usd"].fillna(0),
#     reg_cap_adjust_mil_usd = lambda df : df["reg_cap_adjust_mil_usd"].fillna(0),
# )[["month", "area", "n_projects", "new_registered_cap_mil_usd", "reg_cap_adjust_mil_usd"]]

# if check_values == False:
#     combined_fdi = clean_data(
#         combined_fdi,
#         "area",
#         combined_fdi.columns[2:]
#     )
#     combined_fdi.to_csv("combined_data/combined_fdi.csv", index=False)

In [ ]:
# # Retail sale

# combined_retail = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 12)
#     combined_retail = pd.concat([combined_retail, sheet], axis = 0)

# combined_retail = combined_retail.rename(columns={
#     0:"goods_service",
#     1:"perf_bil_vnd_lag1",
#     2:"est_perf_bil_vnd",
#     3:"accum_bil_vnd",
#     4:"structure_pct",
#     5:"pct_yoy",
#     6:"pct_accum_yoy"
# })[["month","goods_service", "perf_bil_vnd_lag1", "est_perf_bil_vnd", "accum_bil_vnd", "structure_pct", "pct_yoy", "pct_accum_yoy"]]

# if check_values == False:
#     combined_retail = clean_data(
#         combined_retail,
#         "goods_service",
#         combined_retail.columns[2:]
#     )
#     combined_retail.to_csv("combined_data/combined_retail.csv", index=False)

In [ ]:
# # Export

# combined_export = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 13, col_index = [1, 2, 3, 5, 6, 8, 9, 11, 12])
#     combined_export = pd.concat([combined_export, sheet], axis = 0)

# combined_export = combined_export.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_export = clean_data(
#         combined_export,
#         "product",
#         combined_export.columns[2:]
#     )
#     combined_export.to_csv("combined_data/combined_export.csv", index=False)

In [ ]:
# # Import

# combined_import = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 14, col_index = [1, 2, 3, 5, 6, 8, 9, 11, 12])
#     combined_import = pd.concat([combined_import, sheet], axis = 0)

# combined_import = combined_import.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_import = clean_data(
#         combined_import,
#         "product",
#         combined_import.columns[2:]
#     )
#     combined_import.to_csv("combined_data/combined_import.csv", index=False)

In [ ]:
# # CPI

# combined_cpi = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 15).iloc[2:,:]
#     combined_cpi = pd.concat([combined_cpi, sheet], axis = 0)

# combined_cpi = combined_cpi.assign(
#     sector = lambda df : (df[0].ffill() + ": " + df[1].fillna("").astype(str) + " " + df[2].fillna("").astype(str)).str.replace("nan", "")
# )

# combined_cpi = combined_cpi.rename(columns={
#     3:"cpi_base",
#     4:"cpi_yoy",
#     5:"cpi_begin_year",
#     6:"cpi_mom",
#     7:"avg_cpi_yoy"
# })[["month", "sector","cpi_base", "cpi_yoy", "cpi_begin_year", "cpi_mom", "avg_cpi_yoy"]]

# if check_values == False:
#     combined_cpi = clean_data(
#         combined_cpi,
#         "sector",
#         combined_cpi.columns[2:]
#     )
#     combined_cpi.to_csv("combined_data/combined_cpi.csv", index=False)

# clean_data(
#     combined_cpi,
#     "sector",
#     combined_cpi.columns[2:]
# )

In [ ]:
# Carriage of passengers

# combined_car_pas = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 16).iloc[1:,]
#     combined_car_pas = pd.concat([combined_car_pas, sheet], axis = 0)

# combined_car_pas = combined_car_pas.rename(columns={
#     1:"product",
#     2:"est_vol_1000_ton",
#     3:"est_val_mil_usd",
#     5:"est_accum_vol_1000_ton",
#     6:"est_accum_val_mil_usd",
#     8:"yoy_vol_1000_ton",
#     9:"yoy_val_mil_usd",
#     11:"yoy_accum_vol_1000_ton",
#     12:"yoy_accum_val_mil_usd"
# })[["month", "product", "est_vol_1000_ton", "est_val_mil_usd", "est_accum_vol_1000_ton", "est_accum_val_mil_usd", "yoy_vol_1000_ton", "yoy_val_mil_usd", "yoy_accum_vol_1000_ton", "yoy_accum_val_mil_usd"]]

# if check_values == False:
#     combined_car_pas = clean_data(
#         combined_car_pas,
#         "product",
#         combined_car_pas.columns[2:]
#     )
#     combined_car_pas.to_csv("combined_data/combined_car_pas.csv", index=False)


In [33]:
is_parent = combined_car_pas.iloc[:,0].str.startswith("By ", na=False)

combined_car_pas["name"] = (
    (combined_car_pas.iloc[:,0].where(is_parent).ffill()).astype(str) + 
    " - " + 
    combined_car_pas.iloc[:,0].astype(str)
).str.replace(r"nan", "", regex=True).replace(r"^\s-\s", "", regex=True).replace("", None)

In [ ]:
# # International visitors

# combined_visitors = pd.DataFrame()

# for excel_path in monthly_files:
#     sheet = use_columns(excel_path, sheet_index = 18, col_index = [1, 2, 3, 4, 5, 6])
#     combined_visitors = pd.concat([combined_visitors, sheet], axis = 0)

# combined_visitors = combined_visitors.rename(columns={
#     1:"category",
#     2:"arrivals_lag1",
#     3:"arrivals",
#     4:"accum_arrivals",
#     5:"arrivals_yoy",
#     6:"accum_arrivals_yoy"
# })[["month", "category", "arrivals_lag1", "arrivals", "accum_arrivals", "arrivals_yoy", "accum_arrivals_yoy"]]

# if check_values == False:
#     combined_visitors = clean_data(
#         combined_visitors,
#         "category",
#         combined_visitors.columns[2:]
#     )
#     combined_visitors.to_csv("combined_data/combined_visitors.csv", index=False)
